<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
launch_sites = [
    {"Launch Site": "CCAFS LC-40", "Lat": 28.562302, "Long": -80.577356},
    {"Launch Site": "CCAFS SLC-40", "Lat": 28.563197, "Long": -80.576820},
    {"Launch Site": "KSC LC-39A", "Lat": 28.573255, "Long": -80.646895},
    {"Launch Site": "VAFB SLC-4E", "Lat": 34.632834, "Long": -120.610746},
]

# Initialize the map
site_map = folium.Map(location=[28.562302, -80.577356], zoom_start=5)

# Add markers for all launch sites
for site in launch_sites:
    folium.Marker(
        location=[site["Lat"], site["Long"]],
        popup=site["Launch Site"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(site_map)

# Display the map
site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Create a map centered at the first launch site's coordinates
launch_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=5)

# Add markers for each launch site
for _, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(launch_map)

# Display the map
launch_map


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
nasa_center_lat, nasa_center_long = 29.559684, -95.083097

# Create a map centered at NASA Johnson Space Center in Houston, Texas
launch_map = folium.Map(location=[nasa_center_lat, nasa_center_long], zoom_start=10)


We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# NASA Johnson Space Center coordinates
nasa_center_lat, nasa_center_long = 29.559684, -95.083097

# Create a Folium map centered at NASA Johnson Space Center
launch_map = folium.Map(location=[nasa_center_lat, nasa_center_long], zoom_start=10)

# Add a highlighted circle area for NASA Johnson Space Center
folium.Circle(
    location=[nasa_center_lat, nasa_center_long],
    radius=1000,  # Circle radius in meters
    color='blue',
    fill=True,
    fill_color='lightblue',
    popup='NASA Johnson Space Center'
).add_to(launch_map)

# Add a text label for NASA Johnson Space Center
folium.Marker(
    location=[nasa_center_lat, nasa_center_long],
    icon=None,
    popup="NASA Johnson Space Center",
    tooltip="NASA Johnson Space Center",
).add_to(launch_map)

# Display the map
launch_map


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# NASA Johnson Space Center coordinates for map initialization
nasa_center_lat, nasa_center_long = 29.559684, -95.083097

# Create a folium map centered at NASA Johnson Space Center
launch_map = folium.Map(location=[nasa_center_lat, nasa_center_long], zoom_start=5)

# Sample data: Replace this with your actual `launch_sites` DataFrame
launch_sites_data = {
    "Launch Site": ["CCAFS LC-40", "VAFB SLC-4E", "KSC LC-39A", "CCAFS SLC-40"],
    "Lat": [28.562302, 34.632834, 28.573255, 28.563197],
    "Long": [-80.577356, -120.610746, -80.646895, -80.576820],
}
launch_sites_df = pd.DataFrame(launch_sites_data)

# Add a circle and label for each launch site
for _, row in launch_sites_df.iterrows():
    # Add a circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Radius in meters
        color='#008000',  # Circle border color (green)
        fill=True,
        fill_color='#FFFF00',  # Fill color (yellow)
        popup=row['Launch Site']
    ).add_to(launch_map)

    # Add a marker with a label for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>',
        )
    ).add_to(launch_map)

# Display the map
launch_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map
# Filter relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Create a folium map centered at NASA Johnson Space Center
nasa_center_lat, nasa_center_long = 29.559684, -95.083097
launch_map = folium.Map(location=[nasa_center_lat, nasa_center_long], zoom_start=5)

# Add markers for success/failed launches
for _, row in spacex_df.iterrows():
    # Determine marker color based on launch success or failure
    marker_color = 'green' if row['class'] == 1 else 'red'
    marker_icon = 'ok' if row['class'] == 1 else 'remove'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Site: {row['Launch Site']}<br>Status: {'Success' if row['class'] == 1 else 'Failed'}",
        icon=folium.Icon(color=marker_color, icon=marker_icon)
    ).add_to(launch_map)

# Display the map
launch_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [14]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def get_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

# Apply the function to the `class` column and create a new `marker_color` column
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# Display the updated DataFrame
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Create a new column in spacex_df to store marker colors based on the class value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create a Folium map centered at NASA Johnson Space Center
nasa_center_lat, nasa_center_long = 29.559684, -95.083097
launch_map = folium.Map(location=[nasa_center_lat, nasa_center_long], zoom_start=5)

# Add a MarkerCluster object to the map
marker_cluster = MarkerCluster().add_to(launch_map)

# Add markers for all launch records
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=(f"<b>Launch Site:</b> {row['Launch Site']}<br>"
               f"<b>Status:</b> {'Success' if row['class'] == 1 else 'Failed'}"),
        icon=folium.Icon(color=row['marker_color'], icon='info-sign')
    ).add_to(marker_cluster)

# Display the map
launch_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [16]:
import numpy as np

# Function to calculate Cartesian distance
def cartesian_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in kilometers
    x = (lon2 - lon1) * np.cos(np.radians((lat1 + lat2) / 2))
    y = lat2 - lat1
    return R * np.sqrt(x**2 + y**2)

# Ensure unique rows in spacex_df
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Calculate distances between unique launch records
distances = []
for index1, record1 in spacex_df.iterrows():
    for index2, record2 in spacex_df.iterrows():
        if index1 < index2:  # Avoid duplicate pairs and self-comparisons
            distance = cartesian_distance(record1['Lat'], record1['Long'], record2['Lat'], record2['Long'])
            distances.append({
                'From': record1['Launch Site'],
                'To': record2['Launch Site'],
                'Distance (km)': round(distance, 2)
            })

# Create a DataFrame for distances
distances_df = pd.DataFrame(distances)

# Save distances to a CSV file for viewing
distances_df.to_csv("distances_between_launch_sites.csv", index=False)

# Print the DataFrame for distances
print(distances_df)


          From            To  Distance (km)
0  CCAFS LC-40   VAFB SLC-4E      220656.77
1  CCAFS LC-40    KSC LC-39A         395.30
2  CCAFS LC-40  CCAFS SLC-40           6.45
3  VAFB SLC-4E    KSC LC-39A      220260.48
4  VAFB SLC-4E  CCAFS SLC-40      220657.61
5   KSC LC-39A  CCAFS SLC-40         397.29


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [17]:
# Add a MousePosition plugin to display coordinates on the map
mouse_position = MousePosition(
    position='topright',  # Position of the coordinates display
    separator=' : ',      # Separator between latitude and longitude
    empty_string='Unavailable',  # Text displayed when no coordinates are available
    lng_first=True,       # Show longitude before latitude
    num_digits=5,         # Precision of coordinates
    prefix='Coordinates'  # Prefix before the coordinates
)
launch_map.add_child(mouse_position)


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [19]:

# List of launch sites
launch_sites = [
    {"Launch Site": "CCAFS LC-40", "Lat": 28.562302, "Long": -80.577356},
    {"Launch Site": "CCAFS SLC-40", "Lat": 28.563197, "Long": -80.576820},
    {"Launch Site": "KSC LC-39A", "Lat": 28.573255, "Long": -80.646895},
    {"Launch Site": "VAFB SLC-4E", "Lat": 34.632834, "Long": -120.610746},
]

# Initialize map centered at a launch site
site_map = folium.Map(location=[launch_sites[0]['Lat'], launch_sites[0]['Long']], zoom_start=10)

# Add MousePosition plugin
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Add markers for each launch site
for site in launch_sites:
    folium.Marker(
        location=[site["Lat"], site["Long"]],
        popup=f"Launch Site: {site['Launch Site']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Ask user to replace hardcoded coastline coordinates with ones captured via MousePosition
print("Use the map to find the coastline coordinates using MousePosition. Replace the coordinates below.")

# Replace the hardcoded values after noting coordinates from MousePosition
coastline_lat, coastline_lon = 28.56367, -80.57163  # Example

folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Calculate distance between the closest coastline and the launch site
launch_site_lat, launch_site_lon = launch_sites[0]['Lat'], launch_sites[0]['Long']
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Draw a line between the launch site and the coastline point
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="red",
    weight=2.5,
    opacity=1
).add_to(site_map)

# Add a popup showing the distance
folium.Marker(
    location=[(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],
    popup=f"Distance to Coastline: {distance_coastline:.2f} km",
    icon=folium.DivIcon(html=f"<div style='font-size: 12pt'>{distance_coastline:.2f} km</div>")
).add_to(site_map)

# Display the map
site_map


Use the map to find the coastline coordinates using MousePosition. Replace the coordinates below.


In [20]:
# Add a distance marker using DivIcon
distance_marker = folium.Marker(
    location=[(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [21]:
# Coordinates for CCAFS LC-40 launch site and its closest coastline point
launch_lat, launch_lon = 28.562302, -80.577356
coast_lat, coast_lon = 28.5623, -80.56765

# Calculate distance to the coastline
distance_to_coastline = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Add a distance marker using DivIcon
folium.Marker(
    location=[(launch_lat + coast_lat) / 2, (launch_lon + coast_lon) / 2],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 12px; color: black;"><b>{distance_to_coastline:.2f} km</b></div>'
    )
).add_to(launch_map)

# Draw a PolyLine between the launch site and the coastline point
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
    color='blue',
    weight=2
).add_to(launch_map)

# Display the map
launch_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [26]:
# Coordinates for CCAFS LC-40 launch site and its closest coastline point
launch_lat, launch_lon = 28.562302, -80.577356
coast_lat, coast_lon = 28.5623, -80.56765

# Calculate distance to the coastline
distance_to_coastline = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Draw a PolyLine between the launch site and the coastline point and add a distance marker
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
    color='blue',
    weight=2
).add_to(launch_map)

folium.Marker(
    location=[(launch_lat + coast_lat) / 2, (launch_lon + coast_lon) / 2],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 12px; color: black;"><b>{distance_to_coastline:.2f} km</b></div>'
    )
).add_to(launch_map)

# Add markers for railroad, highway, and nearest city
railroad_lat, railroad_lon = 28.56239, -80.57698
highway_lat, highway_lon = 28.56212, -80.57057
city_lat, city_lon = 28.38898, -80.60017

# Calculate distances
distance_to_railroad = calculate_distance(launch_lat, launch_lon, railroad_lat, railroad_lon)
distance_to_highway = calculate_distance(launch_lat, launch_lon, highway_lat, highway_lon)
distance_to_city = calculate_distance(launch_lat, launch_lon, city_lat, city_lon)

# Draw PolyLines and add distance markers
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [railroad_lat, railroad_lon]],
    color='green',
    weight=2
).add_to(launch_map)

folium.Marker(
    location=[(launch_lat + railroad_lat) / 2, (launch_lon + railroad_lon) / 2],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 12px; color: black;"><b>{distance_to_railroad:.2f} km</b></div>'
    )
).add_to(launch_map)

folium.PolyLine(
    locations=[[launch_lat, launch_lon], [highway_lat, highway_lon]],
    color='yellow',
    weight=2
).add_to(launch_map)

folium.Marker(
    location=[(launch_lat + highway_lat) / 2, (launch_lon + highway_lon) / 2],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 12px; color: black;"><b>{distance_to_highway:.2f} km</b></div>'
    )
).add_to(launch_map)

folium.PolyLine(
    locations=[[launch_lat, launch_lon], [city_lat, city_lon]],
    color='red',
    weight=2
).add_to(launch_map)

folium.Marker(
    location=[(launch_lat + city_lat) / 2, (launch_lon + city_lon) / 2],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 12px; color: black;"><b>{distance_to_city:.2f} km</b></div>'
    )
).add_to(launch_map)

# Display the map
launch_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
